In [ ]:
import sys
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import os.path
import matplotlib.image as mpimg


In [ ]:
ddsm_dir = pathlib.Path('/home/Data/mamo/DDSM_png')
labels_dir = '/home/lloprib/proyecto_mam/Mammography/Yolo8Mamo/func_pruebas_lucia/labels/'

In [ ]:
class CustomImageDataset(Dataset):
    def _init_(self, img_dir, labels_dir, nclass, transform=None):
        self.img_dir = img_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.nclass = nclass
        self.img_labels = [f for f in os.listdir(labels_dir) if f.endswith('.txt') ] 

    def _len_(self):
        return len(self.img_labels)

    def _getitem_(self, idx):
        carpetas=self.img_labels[idx].split('.')

        img_path = os.path.join(self.img_dir, carpetas ,self.img_labels[idx].replace('.txt', '.jpg'))  # Assuming image files are .jpg
        image = Image.open(img_path)
        img_width, img_height = image.size
        if self.transform:
            image = self.transform(image)
        
        # Read the label from the corresponding text file
        label_path = os.path.join(self.labels_dir, self.img_labels[idx])
        objeto=[]
        faster_rcnn_boxes = []
        with open(label_path, 'r') as file:
            label_data = file.readlines()
            if label_data:
                for line in label_data:
                    objeto.append(1)
                    coordenadas = list(map(float, line.strip().split()[1:]))
                    coordenadas_yolo=[(max(coordenadas[::2])+min(coordenadas[::2]))/2, 
                                             (max(coordenadas[1::2])+min(coordenadas[1::2]))/2, 
                                             max(coordenadas[::2])-min(coordenadas[::2]),
                                             max(coordenadas[1::2])-min(coordenadas[1::2])]

                    x_center, y_center, width, height = coordenadas_yolo
                    
                    x_min = (x_center - width / 2) * img_width
                    y_min = (y_center - height / 2) * img_height
                    x_max = (x_center + width / 2) * img_width
                    y_max = (y_center + height / 2) * img_height
                    faster_rcnn_boxes.append([x_min, y_min, x_max, y_max])
                target = {
                'boxes': torch.tensor(faster_rcnn_boxes),  
                'labels': torch.tensor(objeto)  
                }
            else:
                 target = {
                'boxes': torch.zeros((0, 4), dtype=torch.float32),
                'labels': torch.zeros((0,), dtype=torch.int64)
                 }
        
        if self.transform:
                    # Apply the same transform to the bounding boxes
                    for i in range(len(target['boxes'])):
                        box = target['boxes'][i]
                        x_min, y_min, x_max, y_max = box
                        box = torch.tensor([
                            x_min / img_width * 800,  # Escala X (xmin)
                            y_min / img_height * 800, # Escala Y (ymin)
                            x_max / img_width * 800,  # Escala X (xmax)
                            y_max / img_height * 800  # Escala Y (ymax)
                        ])
                        target['boxes'][i] = box

        return image, target

transform = T.Compose([#PROBAR A REDIMENSIONAR A 800X800 PARA VER SI MEJORA
    T.Resize(800),        # Redimensiona el lado más pequeño a 256, manteniendo el aspect ratio
    T.CenterCrop((800, 800)),  # Recorta el centro a 256x256
    T.ToTensor(), 
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

    # Update the dataset to use the new transform
train_dataset = CustomImageDataset(img_dir=ddsm_dir, labels_dir=labels_dir, 
                                   nclass=1,transform=transform)